# HW 8: 自駕車接收命令與指揮 (計算理論 / 系統軟體)

2020.11.20

## Tutorial

- [TA placehold]()
- [Regular Operations (05:24)](https://www.youtube.com/watch?v=nIp604p0M8M)
- [Stderr Stdout and Stdin - How to Redirect them - Commands for Linux (05:24)](https://www.youtube.com/watch?v=icuV2CR3Ghg)
  - those function almost are same in windows `PowerShell` and mac `Terminal`

## Objective

- Learn language grammar to create own language which handled by specific application
- Understand regular language and context free language
- Learn useful command-line skill to leverage various system software
- Understand how application leverage system software

## Problem

我們已經能夠將自己腦中的想法在小鴨車用 `Python` 去實作，諸如 `前進`、`亮燈`、`拍照`等  
但每次都要花大量時間使用 `Python` 去做程式設計，是否有更簡單的語言只要寫 `前進`、`亮燈` 等指令，就能實現腦中的想法呢？  
而且簡單的語言也能讓小鴨車初學者更容易上手

同時，在秉持著不重複造輪子的精神下  
讓支援該語言的直譯器透過系統軟體擴充更多功能，並與其他系統上軟體進行合作


## Requirement

實作出小鴨車語 `DuckieDuckie1` 直譯器，並可以接受標準輸入或純文本檔案作為程式碼來源

- 小鴨車語 `DuckieDuckie1` 直譯器
  - 利用正則運算 (Regular Operation) 表達語法  
  - 透過範例與敘述表達語義  
  - **且目前 `DuckieDuckie1` 只支援單句表達式**

- 標準輸入或純文本檔案
  - 只實作從標準輸入讀入 `DuckieDuckie1` 程式碼
  - 利用 輸入重定向 來支援純文本檔案

### 小鴨車語 `DuckieDuckie1`

#### 語法 Syntax

小鴨車語 `DuckieDuckie1` 的正則語法定義為

$$
G = \{ \sum, N, S, P \} \\
G \space \text{is grammar} \\
\sum \space \text{is alphabet} \\
N \space \text{is nonterminal symbol} \\
S \space \text{is start symbol} \\
P \space \text{is production rules}
$$

- `alphabet` 字母集為 `a-z`, `0-9`, `;` 與 空白 ` ` ，亦指終端符號
- `nonterminal symbol` 非終端符號有 `CMD`, `DIGIT`, `SCALAR`, `UNIT`, `DUCKIEDUCKIE1`
- `start symbol` 開始符號為 `DUCKIEDUCKIE1`
- `production rules` 產生規則較為複雜，其中使用到幾個符號
  - 使用三個正則運算： 用 $\cup$ 表示 Union 聯集、用 $\cdot$ 表示 Concat 連接、用 `$*$ 表示 Kleene Star 星號  
  - 使用單引號 $'$ 表達字母集中的終端符號  
  - 沒有單引號也非正則運算表達非終端符號

$$
\begin{align}
\text{CMD} &-> \text{'forward'} \cup \text{'backward'} \cup \text{'left'} \cup \text{'right'} \\
\text{DIGIT} &-> \text{'0'} \cup \text{'1'} 
\cup \text{'2'} \cup \text{'3'} \cup \text{'4'} \cup \text{'5'}
\cup \text{'6'} \cup \text{'7'} \cup \text{'8'} \cup \text{'9'} \\
\text{SCALAR} &-> \text{DIGIT} \cdot \text{DIGIT}^* \\
\text{UNIT} &-> \text{'ms'} \cup \text{'s'} \\
\\
\text{DUCKIEDUCKIE1} &-> \text{CMD} \cdot \text{' '} \cdot \text{' '}^* \cdot \text{SCALAR} \cdot \text{UNIT} \cdot \text{';'}
\end{align}
$$

其對應的正則表達式 Regular Expression 為

```
(?P<CMD>forward|backward|left|right)[ ][ ]*(?P<SCALAR>\d\d*)(?P<UNIT>ms|s);
```


#### 語義 Semantics

`DuckieDuckie1` 會被分成三個字串，分別是 `CMD`、`SCALAR`、`UNIT`

- `CMD` 對應到小鴨車應該要有的行為
- `SCALAR` 對應到 `CMD` 的參數，與 `UNIT` 配合表示不同量級的參數
- `UNIT` 對應到 `SCALAR` 的單位

其對應語義為：控制小鴨車進行 `CMD` 行為，持續時間為 `SCALAR`，時間單位為 `UNIT`

舉例而言，當 `CMD='forward', SCALAR='1', UNIT='s'`  
對應語義即為，小鴨車前進一秒


## Test Example

小鴨車語 `DuckieDuckie1` 的使用範例與行為

有一個檔案 `main.duckie`
```duckieduckie
forward 1500ms;
```

透過命令列 `python duckieduckie1.py < main.duckie` 使用輸入重定向
> 但在小鴨車環境上的話，已經是 `bash` 所以可以直接使用輸入重定向

預期會看到小鴨車的行為是：

```
前進 1500 毫秒，最後停止
```


## Steps 實作步驟

僅供參考，並非寫出作業的唯一方法  
只是其中一種方法

### Step1: 從標準輸入 `STDIN` 取得小鴨車語 `DuckieDuckie1` 的純文本

其實第一個作業大家就都已經會了，就是使用 `input`  
關於文件可以看 [python3.6 官方 builtin `input` 參考文件](https://docs.python.org/3.6/library/functions.html#input)

但這邊要處理當遇到 `EOF` (end of file) 檔案結尾時，會拋出來的 `EOFError`  
要在使用終端輸入時，產生 `EOF` 要根據不同作業系統

- windows 使用 `Ctrl + Z` 然後按下 `enter`
- mac & linux 使用 `Ctrl + D`

下面的程式碼展示了如何處理 `EOF`，同時該程式碼有部分 `cat` 指令的效果

In [1]:
%%writefile ./cat.py
# use to save all input text
whole_input: str = ""
while True:
    try:
        # wait STDIN until met newline or EOF
        line: str = input('')
        # because input will consume newline '\n'
        whole_input += line + '\n'
    # when EOFError happend then exit loop
    except EOFError:
        break

print(whole_input, end='')

Overwriting ./cat.py


### Step2: 利用輸入重定向，餵入小鴨車語 `DuckieDuckie1` 程式碼檔案

在終端畫面，用 `<` 代表將輸入重定向，使 `STDIN` 被指到一個檔案
 
- windows 可以用 `powershell` 或其他支援的系統軟體
- mac 可以用 `terminal` 或其他支援的系統軟體
- linux 可以用 `sh` 或 `bash` 或 `zsh` 或其他支援的系統軟體

先準備小鴨車語 `DuckieDuckie1` 程式碼檔案

In [2]:
%%writefile ./main.duckie
forward 1500ms;

Overwriting ./main.duckie


剛剛 Step1 的範例程式碼，可以直接產生如 `cat` 的效果

In [3]:
! $sys.executable ./cat.py < ./main.duckie
# or python ./cat.py < ./main.duckie

forward 1500ms;


### Step3: 使用 正則表達式 Regular Expression 剖析語言，辨識是否符合語法

因 `Python` 本身具有 `re` 可以理解 正則表達式 並且解析  
所以我們可以直接利用它完成辨識語法的功能，關於文件可以看 [python3.6 官方 re 參考文件](https://docs.python.org/3.6/library/re.html)

如果遇到不符合語法的小鴨車語 `DuckieDuckie1` 請結束並提示語法錯誤有錯

In [4]:
import re

# create parser through regular expression (grammar)
parser = re.compile("(?P<CMD>forward|backward|left|right)[ ][ ]*(?P<SCALAR>\d\d*)(?P<UNIT>ms|s);")

# one line legal `DuckieDuckie1` expression
match = parser.match("forward 1500ms;\n")

# one line illegal `DuckieDuckie1` expression
match = parser.match("goforward 3000ns;\n")

### Step4: 透過剖析後的結果，執行對應的語義

透過使用 正則表達式 的群組功能，我們可以取出它所剖析出來的個別字串  
並根據小鴨車語 `DuckieDuckie1` 的語義，去控制小鴨車

> 因為需要控制小鴨車，會使用到 `libs.car_control`  
> 請記得將程式碼放置於 `duckiebot_cs_zoo` 目錄下


In [5]:
match = parser.match("backward 500ms;\n")

# get parsed group `CMD`, `SCALAR`, `UNIT`
cmd = match.group('CMD')
scalar = match.group('SCALAR')
unit = match.group('UNIT')
print('I am duckiebot')
print('do ' + cmd + ' for ' + scalar + ' times in ' + unit + ' unit.')

I am duckiebot
do backward for 500 times in ms unit.


### Step5: 應用寫出來的小鴨車語 `DuckieDuckie1` 直譯器

準備小鴨車語 `DuckieDuckie1` 程式碼

```
backward 2s;
```

輸入你的直譯器

```
$ python yourcode.py < filename
```

看到小鴨車依照語義，進行後退兩秒的動作

## Notice

- 因需要使用到 `duckiebot_cs_zoo/libs` 目錄下的函式庫，請將程式碼放置於 `duckiebot_cs_zoo` 目錄下
- deadline: 2020.11.27 before class

## Reference

- [python3.6 官方 builtin `input` 參考文件](https://docs.python.org/3.6/library/functions.html#input)
- [python3.6 官方 re 參考文件](https://docs.python.org/3.6/library/re.html)
- [regex 101](https://regex101.com/)
- [video: Introduction to Theory of Computation](https://www.youtube.com/watch?v=58N2N7zJGrQ&list=PLBlnK6fEyqRgp46KUv4ZY69yXmpwKOIev)
- [video: What Is System Software ? | Functions And Types Of System Software](https://www.youtube.com/watch?v=Ts8hvn198mM)